In [1]:
import numpy as np 
import pandas as pd
import os
import datetime

In [2]:
# Importing the csv file and analyzing, understanding each coloumn
data = pd.read_csv('Data/AMZN.csv', thousands=',')
data.columns

Index(['Date', 'Close Price', 'Open Price', 'High Price', 'Low Price',
       'Total Traded Quantity'],
      dtype='object')

In [3]:
data.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
0,"Oct 24, 2019",1780.78,1770.78,1787.97,1760.78,5.20M
1,"Oct 23, 2019",1762.17,1761.30,1770.05,1742.00,2.19M
2,"Oct 22, 2019",1765.73,1788.15,1789.78,1762.00,2.23M
3,"Oct 21, 2019",1785.66,1769.66,1785.88,1765.00,2.22M
4,"Oct 18, 2019",1757.51,1787.80,1793.98,1749.20,3.37M


In [4]:
data.tail()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
498,"Oct 31, 2017",1105.28,1109.00,1110.54,1101.12,3.48M
499,"Oct 30, 2017",1110.85,1095.01,1122.79,1093.56,6.61M
500,"Oct 27, 2017",1100.95,1058.14,1105.58,1050.55,16.57M
501,"Oct 26, 2017",972.43,980.33,982.90,968.55,5.62M
502,"Oct 25, 2017",972.91,978.00,984.44,966.24,3.03M


In [5]:
data.describe()

,Close Price,Open Price,High Price,Low Price
count,503.000000,503.000000,503.000000,503.000000
mean,1657.415964,1658.401809,1674.438171,1638.693459
std,237.374706,238.679074,238.361157,237.377795
min,972.430000,978.000000,982.900000,966.240000
25%,1534.030000,1541.495000,1559.095000,1509.605000
50%,1698.750000,1702.950000,1715.270000,1680.890000
75%,1832.310000,1832.990000,1845.560000,1816.800000
max,2039.510000,2038.110000,2050.500000,2013.000000


In [6]:
#Min, Max and Mean price for the last 90 days.
print ("Max Price :" , data.tail(90)['Close Price'].max())
print ("Min Price :" , data.tail(90)['Close Price'].min())
print ("Mean Price :" , data.tail(90)['Close Price'].mean())

Max Price : 1537.64
Min Price : 972.43
Mean Price : 1261.3811111111113


In [7]:
#Analysing data types of the coloumns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 6 columns):
Date                     503 non-null object
Close Price              503 non-null float64
Open Price               503 non-null float64
High Price               503 non-null float64
Low Price                503 non-null float64
Total Traded Quantity    503 non-null object
dtypes: float64(4), object(2)
memory usage: 23.7+ KB


In [8]:
#Converting Total Traded Quantity from object to float64
data['Total Traded Quantity'] = (data['Total Traded Quantity'].str.replace('M','').astype(float) * 1000000)
#datetime64[ns] maps to either <M8[ns] or >M8[ns] depending on the endian-ness of the machine
data['Date'] = pd.to_datetime(data['Date'])
data['Date'].dtype

dtype('<M8[ns]')

In [9]:
data['Date'].max()-data['Date'].min()

Timedelta('729 days 00:00:00')

In [10]:
datatype= data.dtypes
datatype

Date                     datetime64[ns]
Close Price                     float64
Open Price                      float64
High Price                      float64
Low Price                       float64
Total Traded Quantity           float64
dtype: object

In [11]:
#Calculating the monthwise Volume Weighted Average Price (VWAP)
data['Month'] = pd.DatetimeIndex(data['Date']).month
data['Year'] = pd.DatetimeIndex(data['Date']).year
data['VWAP'] = (data['Close Price'] * data['Total Traded Quantity']).cumsum() / data['Total Traded Quantity'].fillna(0).cumsum()
data_vwap = data[['Month','Year','VWAP']]
group = data_vwap.groupby(['Month','Year'])
group.first()

VWAP
Month Year             
1     2018  1696.770754
      2019  1802.113970
2     2018  1715.464550
      2019  1825.607774
3     2018  1727.251500
      2019  1845.508315
4     2018  1747.607581
      2019  1841.533792
5     2018  1754.481397
      2019  1830.660877
6     2018  1757.575470
      2019  1832.156343
7     2018  1755.377777
      2019  1783.015274
8     2018  1745.892721
      2019  1773.015052
9     2018  1730.125221
      2019  1747.291733
10    2017  1651.013549
      2018  1725.628039
      2019  1780.780000
11    2017  1667.741061
      2018  1740.458323
12    2017  1680.972117
      2018  1774.731674

In [12]:
#OHLC Average Price over the last N days.
x=data.iloc[:,1:5].sum(axis=1)
data['OHLC Avg Price']=x/4

def avg_price(N):
    return (data['OHLC Avg Price'].tail(N).sum())/N

print("Average prices for last N days are as follows:")
print("Last 1 week",avg_price(5))
print("Last 2 weeks",avg_price(10))
print("Last 1 month",avg_price(20))
print("Last 3 months",avg_price(60))
print("Last 6 months",avg_price(120))
print("Last 1 year",avg_price(240))

Average prices for last N days are as follows:
Last 1 week 1048.4585
Last 2 weeks 1078.2185000000002
Last 1 month 1103.9773750000002
Last 3 months 1169.4862083333333
Last 6 months 1319.4782500000001
Last 1 year 1547.3974375


In [13]:
#Loss/Profit percentage over the last N days.
def prof_loss(N):
    difference = (data['Close Price'].tail(N).iloc[N-1] - data['Close Price'].tail(N).iloc[0])
    if difference < 0 :
        loss = -(difference)
        loss_perc = (loss/data['Close Price'].tail(N).iloc[N-1])*100
        return loss_perc
    if difference > 0 :
        prof = difference
        prof_perc = (prof/data['Close Price'].tail(N).iloc[N-1])*100
        return prof_perc
print("Loss/Profit percentage for last N days are as follows:")
print("Last 1 week",prof_loss(5))
print("Last 2 weeks",prof_loss(10))
print("Last 1 month",prof_loss(20))
print("Last 3 months",prof_loss(60))
print("Last 6 months",prof_loss(120))
print("Last 1 year",prof_loss(240))

Loss/Profit percentage for last N days are as follows:
Last 1 week 13.605575027494837
Last 2 weeks 15.444388484032451
Last 1 month 17.121830385133265
Last 3 months 36.42680206802274
Last 6 months 57.03816385893864
Last 1 year 91.63334738053881


In [14]:
#Obtaining the daily percentage change of the stocks
data['Day_Perc_Change'] = data['Close Price'].pct_change().fillna(0)
data

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,VWAP,OHLC Avg Price,Day_Perc_Change
0,2019-10-24,1780.78,1770.78,1787.97,1760.78,5200000.0,10,2019,1780.780000,1775.0775,0.000000
1,2019-10-23,1762.17,1761.30,1770.05,1742.00,2190000.0,10,2019,1775.264993,1758.8800,-0.010450
2,2019-10-22,1765.73,1788.15,1789.78,1762.00,2230000.0,10,2019,1773.054699,1776.4150,0.002020
3,2019-10-21,1785.66,1769.66,1785.88,1765.00,2220000.0,10,2019,1775.418193,1776.5500,0.011287
4,2019-10-18,1757.51,1787.80,1793.98,1749.20,3370000.0,10,2019,1771.450368,1772.1225,-0.015764
...,...,...,...,...,...,...,...,...,...,...,...
498,2017-10-31,1105.28,1109.00,1110.54,1101.12,3480000.0,10,2017,1651.013549,1106.4850,0.001450
499,2017-10-30,1110.85,1095.01,1122.79,1093.56,6610000.0,10,2017,1649.525582,1105.5525,0.005039
500,2017-10-27,1100.95,1058.14,1105.58,1050.55,16570000.0,10,2017,1645.763425,1078.8050,-0.008912
501,2017-10-26,972.43,980.33,982.90,968.55,5620000.0,10,2017,1644.200870,976.0525,-0.116736


In [15]:
#Obtaining a trend coloumn with the following range
if ((data['Day_Perc_Change'] >= -0.5) & (data['Day_Perc_Change'] <= 0.5)).all():
    data['Trend'] = 'Slight or No change'
if ((data['Day_Perc_Change'] >= 0.5) & (data['Day_Perc_Change'] <= 1)).all():
    data['Trend'] = 'Slight positive'
if ((data['Day_Perc_Change'] <= -0.5) & (data['Day_Perc_Change'] >= -1)).all():
    data['Trend'] = 'Slight negative'
if ((data['Day_Perc_Change'] >= 1) & (data['Day_Perc_Change'] <= 3)).all():
    data['Trend'] = 'Positive' 
if ((data['Day_Perc_Change'] <= -1) & (data['Day_Perc_Change'] >= -3)).all():
    data['Trend'] = 'Negative'
if ((data['Day_Perc_Change'] >= 3) & (data['Day_Perc_Change'] <= 7)).all():
    data['Trend'] = 'Among top gainers'
if ((data['Day_Perc_Change'] <= -3) & (data['Day_Perc_Change'] >= -7)).all():
    data['Trend'] = 'Among top losers'
if (data['Day_Perc_Change'] > 7).all():
    data['Trend'] = 'Bull run' 
if (data['Day_Perc_Change'] < -7).all():
    data['Trend'] = 'Bear drop' 
data

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,VWAP,OHLC Avg Price,Day_Perc_Change,Trend
0,2019-10-24,1780.78,1770.78,1787.97,1760.78,5200000.0,10,2019,1780.780000,1775.0775,0.000000,Slight or No change
1,2019-10-23,1762.17,1761.30,1770.05,1742.00,2190000.0,10,2019,1775.264993,1758.8800,-0.010450,Slight or No change
2,2019-10-22,1765.73,1788.15,1789.78,1762.00,2230000.0,10,2019,1773.054699,1776.4150,0.002020,Slight or No change
3,2019-10-21,1785.66,1769.66,1785.88,1765.00,2220000.0,10,2019,1775.418193,1776.5500,0.011287,Slight or No change
4,2019-10-18,1757.51,1787.80,1793.98,1749.20,3370000.0,10,2019,1771.450368,1772.1225,-0.015764,Slight or No change
...,...,...,...,...,...,...,...,...,...,...,...,...
498,2017-10-31,1105.28,1109.00,1110.54,1101.12,3480000.0,10,2017,1651.013549,1106.4850,0.001450,Slight or No change
499,2017-10-30,1110.85,1095.01,1122.79,1093.56,6610000.0,10,2017,1649.525582,1105.5525,0.005039,Slight or No change
500,2017-10-27,1100.95,1058.14,1105.58,1050.55,16570000.0,10,2017,1645.763425,1078.8050,-0.008912,Slight or No change
501,2017-10-26,972.43,980.33,982.90,968.55,5620000.0,10,2017,1644.200870,976.0525,-0.116736,Slight or No change


In [16]:
#Finding the average and median values of Total Traded Quantity for each type of trend.
xmean= data.groupby(data.Trend).mean()['Total Traded Quantity']
xmedian= data.groupby(data.Trend).median()['Total Traded Quantity']

print(xmean)
print(xmedian)

Trend
Slight or No change    4.820656e+06
Name: Total Traded Quantity, dtype: float64
Trend
Slight or No change    4270000.0
Name: Total Traded Quantity, dtype: float64


In [17]:
data.to_csv('Data/week2.csv')